<a href="https://colab.research.google.com/github/BRIJNANDA1979/CNN-Sentinel/blob/master/CNN_Edusat_Data_8_classes_15_May_2021_Version_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Created on Thu May 13 03:02:04 2021

@author: BRIJB
"""

from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import the needed packages
import tensorflow as tf
from keras import losses 
from keras import optimizers 
from keras import metrics 
import matplotlib.pyplot as plt
import matplotlib.image as img
import tensorflow.keras as keras
from keras.preprocessing import image
import numpy as np
from keras.models import model_from_json
import os

batch_size = 30

# define and move to dataset directory
datasetdir = "/content/drive/MyDrive/Data/Sentinel 2 Data for CNN/EuroSAT/2750"
import os
os.chdir(datasetdir)



# shortcut to the ImageDataGenerator class
ImageDataGenerator = keras.preprocessing.image.ImageDataGenerator
gen = ImageDataGenerator()
iterator = gen.flow_from_directory(
    os.getcwd(), 
    target_size=(256,256), 
    classes=('Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential','SeaLake')
    
)
# we can guess that the iterator has a next function, 
# because all python iterators have one. 
batch = iterator.next()
print(len(batch))
print(type(batch[0]))
print(batch[0].shape)
print(batch[0].dtype)
#print(batch[0].max())
#print(batch[1].shape)
#print(batch[1].dtype)
#print(type(batch[1]))
#the first element is an array of 32 images with 64X64 pixels, and 3 color channels, encoded as floats in the range 0 to 255
#The second element contains the 32 corresponding labels.


Found 16579 images belonging to 8 classes.
2
<class 'numpy.ndarray'>
(32, 256, 256, 3)
float32


In [2]:
batch = iterator.next()
print(len(batch))
print(type(batch[1]))
print(batch[1].shape)
print(batch[1].dtype)

2
<class 'numpy.ndarray'>
(32, 8)
float32


In [ ]:
#Augmentation by Flipping images
#Now, let's make the transformation a bit more complex. This time, the ImageDataGenerator will flip, zoom, and rotate the images on a random basis   
imgdatagen = ImageDataGenerator(
    rescale = 1/255., 
    horizontal_flip = True, 
    zoom_range = 0.3, 
    rotation_range = 15.,
    validation_split = 0.1,
)
batch_size = 30
height, width = (256,256)
train_dataset = imgdatagen.flow_from_directory(
    os.getcwd(),
    target_size = (height, width), 
    classes = ('Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential','SeaLake'),
    batch_size = batch_size,
    subset = 'training'
)

val_dataset = imgdatagen.flow_from_directory(
    os.getcwd(),
    target_size = (height, width), 
    classes = ('Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential','SeaLake'),
    batch_size = batch_size,
    subset = 'validation'
)
model = keras.models.Sequential()

initializers = {
    
}
model.add( 
    keras.layers.Conv2D(
        24, 5, input_shape=(256,256,3), 
        activation='relu', 
    )
)
model.add( keras.layers.MaxPooling2D(2) )
model.add( 
    keras.layers.Conv2D(
        48, 5, activation='relu', 
    )
)
model.add( keras.layers.MaxPooling2D(2) )
model.add( 
    keras.layers.Conv2D(
        96, 5, activation='relu', 
    )
)
model.add( keras.layers.Flatten() )
model.add( keras.layers.Dropout(0.2) )

model.add( keras.layers.Dense(
    2, activation='softmax',
    )
)

model.summary()
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.RMSprop(lr=0.001),
              metrics=['acc'])
model.fit_generator(
    train_dataset, 
    validation_data = val_dataset,
    workers=10,
    epochs=100,
)

# serialize model to JSON https://machinelearningmastery.com/save-load-keras-deep-learning-models/
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")


Found 14922 images belonging to 8 classes.
Found 1657 images belonging to 8 classes.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 252, 252, 24)      1824      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 126, 126, 24)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 122, 122, 48)      28848     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 61, 61, 48)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 57, 57, 96)        115296    
_________________________________________________________________
flatten (Flatten)            (None, 311904)            0         
_____________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
